In [1]:
import dask.dataframe as dd
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from pathlib import Path

In [2]:
base_dir = Path.home()
manipulated_data_directory = base_dir / "NaMinhaRua" / "Ficheiros de dados Manipulados"
df = pd.read_csv(manipulated_data_directory/ 'GOPI&IPMA_MERGED_24h_Subseccao_final.csv')
df.head(2)



,dt_registo,Subseccao,Freguesia,BNFDRB,CA,CDPC2RD,CDRD,CDEEP,DDP,EOVRDJOPANVP,...,RRPDR,SOOLA,ÁAORM,Fim_de_Semana,Feriado,Inverno,Primavera,Verao,Outono,Chuva
0,2018-01-01,11066601201,Santo António,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,NaN
1,2018-01-01,11060200605,Alcântara,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,NaN


In [3]:
# Find which "Freguesia" the "Subseccao" belongs to
# Definir a subseção de interesse
subseccao_interesse = '11063301505'
df['Subseccao'] = df['Subseccao'].astype(str).str.strip()

# Double-check the subsection
if subseccao_interesse in df['Subseccao'].values:
    freguesia = df.loc[df['Subseccao'] == subseccao_interesse, 'Freguesia'].values
    if len(freguesia) > 0:
        print(f"A freguesia correspondente à subseção {subseccao_interesse} é {freguesia[0]}.")
    else:
        print(f"Não foi encontrada a freguesia para a subseção {subseccao_interesse}.")
else:
    print(f"A subseção {subseccao_interesse} não está presente no DataFrame.")



A freguesia correspondente à subseção 11063301505 é Olivais.


In [4]:
# Convert 'Chuva' column to numeric
df['Chuva'] = pd.to_numeric(df['Chuva'], errors='coerce')

# Count the number of days in which the rain was greater than 10
dias_chuva_maior_que_10 = (df['Chuva'] > 10).sum()
print(f"Número de dias em que a chuva foi maior que 10: {dias_chuva_maior_que_10}")

Número de dias em que a chuva foi maior que 10: 1261


In [5]:
dia_max_precip = df.loc[df['Chuva'].idxmax()]

print(dia_max_precip)

dt_registo          2020-10-19
Subseccao          11066600601
Freguesia        Santo António
BNFDRB                   False
CA                       False
CDPC2RD                  False
CDRD                     False
CDEEP                    False
DDP                      False
EOVRDJOPANVP              True
EA                       False
FDDDCDPC2ROSM            False
G                        False
LDVPDDPVELDVP            False
OIEVPER                  False
PED                      False
RNADRDDRSU               False
RSRPDP                   False
RJPDR                    False
RMPDR                    False
RRPDR                    False
SOOLA                    False
ÁAORM                    False
Fim_de_Semana            False
Feriado                  False
Inverno                  False
Primavera                False
Verao                    False
Outono                    True
Chuva                48.416667
Name: 45695, dtype: object


In [5]:
unique_subseccao_count = df['Subseccao'].nunique()

print(unique_subseccao_count)
# Print each unique value present in the 'Subseccao' column
unique_subseccao_values = df['Subseccao'].unique()

print("Valores únicos na coluna 'Subseccao':")
for value in unique_subseccao_values:
    print(value)

344
Valores únicos na coluna 'Subseccao':
11066601201
11060200605
11063301908
11063301802
11066600102
11063301002
11060200705
11063302305
11063300905
11063302506
11063300603
11066601103
11066601109
11066601207
11060201504
11066600907
11066601001
11063300601
11060200401
11063302510
11066601210
11060200607
11066600203
11060200404
11063302606
11066600402
11066600802
11060200801
11066600302
11066600803
11060200701
11066601004
11066600204
11066600201
11063302512
11060201204
11066601206
11066600906
11066601211
11063301003
11063300504
11060201304
11066601410
11063302207
11066600502
11063302210
11066601202
11060200102
11063301610
11063300303
11063301502
11060201101
11066600503
11060200609
11066601505
11060200608
11060201408
11063302004
11066601102
11063301903
11066601501
11066600307
11060201502
11063302002
11066601204
11066601306
11063301804
11063302003
11063301907
11066600908
11066601409
11066600401
11063301901
11066600701
11063301608
11066601304
11060200303
11066600301
11063302605
1106020040

In [6]:
unique_subseccao_count = df['Freguesia'].nunique()

print(unique_subseccao_count)

3


In [7]:
num_linhas = df.shape[0]

print(f"Número de linhas: {num_linhas}")
print(df.dtypes)

Número de linhas: 102081
dt_registo        object
Subseccao          int64
Freguesia         object
BNFDRB              bool
CA                  bool
CDPC2RD             bool
CDRD                bool
CDEEP               bool
DDP                 bool
EOVRDJOPANVP        bool
EA                  bool
FDDDCDPC2ROSM       bool
G                   bool
LDVPDDPVELDVP       bool
OIEVPER             bool
PED                 bool
RNADRDDRSU          bool
RSRPDP              bool
RJPDR               bool
RMPDR               bool
RRPDR               bool
SOOLA               bool
ÁAORM               bool
Fim_de_Semana       bool
Feriado             bool
Inverno             bool
Primavera           bool
Verao               bool
Outono              bool
Chuva            float64
dtype: object


In [6]:
num_nan_chuva = df['Chuva'].isna().sum()
print(f"Número de valores NaN na coluna 'Chuva': {num_nan_chuva}")

# Remover as linhas onde 'Chuva' é NaN
df.dropna(subset=['Chuva'], inplace=True)

Número de valores NaN na coluna 'Chuva': 19324


In [7]:
# Convert 'Chuva' column to numeric
df['Chuva'] = pd.to_numeric(df['Chuva'], errors='coerce')

# Count the number of days in which the rain was greater than 10
dias_chuva_maior_que_10 = (df['Chuva'] > 10).sum()
print(f"Número de dias em que a chuva foi maior que 10: {dias_chuva_maior_que_10}")

Número de dias em que a chuva foi maior que 10: 1261


In [8]:
num_linhas = df.shape[0]

print(f"Número de linhas: {num_linhas}")

Número de linhas: 82757


In [9]:
max_chuva = df['Chuva'].max()

# Show the results
print(f"O valor máximo para 'Chuva' é: {max_chuva}")

O valor máximo para 'Chuva' é: 48.41666666666666


In [10]:
# Create binary columns for each "Subseccao"
Subsecs_dummies = pd.get_dummies(df['Subseccao'])

# Convert values from 1/0 to True/False
Subsecs_dummies = Subsecs_dummies.astype(bool)

# Concatenate the new columns to the original dataframe
df = pd.concat([df, Subsecs_dummies], axis=1)

# Remove the original 'Freguesia' column
df = df.drop('Subseccao', axis=1)

# Display the resulting dataframe
print(df.head())

       dt_registo      Freguesia  BNFDRB     CA  CDPC2RD   CDRD  CDEEP    DDP  \
15290  2018-12-17  Santo António   False  False    False  False  False   True   
15291  2018-12-17        Olivais   False  False    False  False  False  False   
15292  2018-12-17        Olivais   False  False    False  False  False  False   
15293  2018-12-17      Alcântara   False  False    False  False  False  False   
15294  2018-12-17  Santo António   False  False    False  False  False  False   

       EOVRDJOPANVP     EA  ...  11066601406  11066601407  11066601408  \
15290         False  False  ...        False        False        False   
15291         False  False  ...        False        False        False   
15292         False  False  ...        False        False        False   
15293         False  False  ...        False        False        False   
15294         False   True  ...        False        False        False   

       11066601409  11066601410  11066601501  11066601502  110666015

In [11]:
df.head(2)

,dt_registo,Freguesia,BNFDRB,CA,CDPC2RD,CDRD,CDEEP,DDP,EOVRDJOPANVP,EA,...,11066601406,11066601407,11066601408,11066601409,11066601410,11066601501,11066601502,11066601503,11066601504,11066601505
15290,2018-12-17,Santo António,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
15291,2018-12-17,Olivais,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
#Create new data model - with association rules for Subsection
df.to_csv(manipulated_data_directory/ 'gopidatasetmodel1_SUBSECCAO.csv',sep=',', index=False)


In [13]:
df = pd.read_csv(manipulated_data_directory/ 'gopidatasetmodel1_SUBSECCAO.csv')

In [14]:
count_chuva_maior_10 = df[df['Chuva'] > 10].shape[0]

print(count_chuva_maior_10)

1261


In [15]:
print(df.columns)

Index(['dt_registo', 'Freguesia', 'BNFDRB', 'CA', 'CDPC2RD', 'CDRD', 'CDEEP',
       'DDP', 'EOVRDJOPANVP', 'EA',
       ...
       '11066601406', '11066601407', '11066601408', '11066601409',
       '11066601410', '11066601501', '11066601502', '11066601503',
       '11066601504', '11066601505'],
      dtype='object', length=373)


In [16]:
# fpgrowth rules analysis for "Susbseccao"



for col in df.select_dtypes(include='number').columns:
           df[col] = df[col].astype(bool)

# Define relevant columns for FP-Growth analysis
relevant_columns = [
    'BNFDRB', 'CA', 'CDPC2RD', 'CDRD', 'CDEEP', 'DDP', 'EOVRDJOPANVP', 'EA', 
    'FDDDCDPC2ROSM', 'G', 'LDVPDDPVELDVP', 'OIEVPER', 'PED', 'RNADRDDRSU', 
    'RSRPDP', 'RJPDR', 'RMPDR', 'RRPDR', 'SOOLA', 'ÁAORM', 'Fim_de_Semana', 
    'Feriado', 'Inverno', 'Outono', 'Primavera', 'Verao', '11066601201', '11060200605', '11063301908', '11063301802', '11066600102',
    '11063301002', '11060200705', '11063302305', '11063300905', '11063302506',
    '11063300603', '11066601103', '11066601109', '11066601207', '11060201504',
    '11066600907', '11066601001', '11063300601', '11060200401', '11063302510',
    '11066601210', '11060200607', '11066600203', '11060200404', '11063302606',
    '11066600402', '11066600802', '11060200801', '11066600302', '11066600803',
    '11060200701', '11066601004', '11066600204', '11066600201', '11063302512',
    '11060201204', '11066601206', '11066600906', '11066601211', '11063301003',
    '11063300504', '11060201304', '11066601410', '11063302207', '11066600502',
    '11063302210', '11066601202', '11060200102', '11063301610', '11063300303',
    '11063301502', '11060201101', '11066600503', '11060200609', '11066601505',
    '11060200608', '11060201408', '11063302004', '11066601102', '11063301903',
    '11066601501', '11066600307', '11060201502', '11063302002', '11066601204',
    '11066601306', '11063301804', '11063302003', '11063301907', '11066600908',
    '11066601409', '11066600401', '11063301901', '11066600701', '11063301608',
    '11066601304', '11060200303', '11066600301', '11063302605', '11060200405',
    '11063300311', '11063300401', '11066601407', '11063300801', '11063302509',
    '11060200502', '11066601403', '11063300502', '11063301801', '11060201409',
    '11066601305', '11063301805', '11063301707', '11063301702', '11063300501',
    '11060201305', '11066601101', '11060201501', '11066600205', '11063302402',
    '11063300202', '11063302505', '11063302203', '11060201411', '11066600103',
    '11066600702', '11063301620', '11060200305', '11066600306', '11060200603',
    '11063301806', '11063301504', '11063302106', '11060200901', '11066600902',
    '11063301704', '11060200301', '11060200302', '11063301909', '11063301101',
    '11063301803', '11066601002', '11066600903', '11063300703', '11063300503',
    '11063301613', '11063300103', '11063301501', '11060201403', '11066600110',
    '11066600801', '11066601405', '11066600901', '11063302202', '11060201102',
    '11066600304', '11063301705', '11060200902', '11063302602', '11063301102',
    '11063300101', '11063301612', '11063300204', '11060201104', '11066601401',
    '11066600108', '11066600105', '11063300305', '11066600501', '11066600602',
    '11063301904', '11066600206', '11060200402', '11060201412', '11060201301',
    '11066601208', '11063302406', '11063302601', '11060200503', '11063301402',
    '11066601005', '11063302303', '11063301505', '11066600704', '11066601107',
    '11060201002', '11063302306', '11063300102', '11060201402', '11066601007',
    '11063301906', '11063302507', '11060200101', '11066600101', '11060200201',
    '11060200202', '11066601108', '11063301407', '11063301607', '11060200702',
    '11063300203', '11063301706', '11066601003', '11060200504', '11060201103',
    '11066601503', '11063302502', '11060201201', '11060200406', '11063300307',
    '11063300802', '11066601502', '11066601104', '11066600107', '11063300201',
    '11063300308', '11060200802', '11063302201', '11063302302', '11066600202',
    '11060200601', '11063300402', '11066600305', '11060201401', '11060201003',
    '11063300301', '11063301201', '11060200604', '11063301611', '11063301604',
    '11060201503', '11066601209', '11060201309', '11063302304', '11063301619',
    '11060201202', '11060201307', '11063301703', '11063300902', '11063300704',
    '11063301616', '11063300304', '11063302101', '11066600603', '11063300312',
    '11063302107', '11063300702', '11063300903', '11066601106', '11066600904',
    '11063302001', '11063301307', '11063302503', '11066600109', '11063302511',
    '11063301617', '11063301403', '11066600104', '11060200501', '11063300602',
    '11066601203', '11066600804', '11063301905', '11063301701', '11063301303',
    '11063302504', '11066601302', '11063301621', '11060200704', '11060201308',
    '11063302501', '11066601301', '11066601404', '11066600601', '11063301404',
    '11060201302', '11066601008', '11066600303', '11066601402', '11063302604',
    '11063301601', '11063301202', '11063301406', '11060200306', '11063302508',
    '11066601105', '11060200204', '11063301001', '11060200610', '11060200611',
    '11066601504', '11063300403', '11063300904', '11063302206', '11066601303',
    '11063301302', '11063300309', '11063301305', '11063300302', '11066600605',
    '11060200403', '11066601006', '11063301807', '11063302204', '11063302404',
    '11066600604', '11063300701', '11060200606', '11060200203', '11063301602',
    '11063302104', '11060200304', '11063301304', '11063301004', '11060201306',
    '11063302105', '11063302603', '11063301605', '11066600703', '11063302607',
    '11066601212', '11063302301']

# Keep only the relevant columns in the basket
basket = df[relevant_columns]

# Optimize data types by converting boolean columns to int8 (takes less memory)
basket = basket.astype('int8')

# Apply fpgrowth algorithm with minimum support
start_time = time.time()
frequent_itemsets = fpgrowth(basket, min_support=0.001, use_colnames=True)
#0.0027
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
fp_growth_exec_time = time.time() - start_time
print("Itemsets frequentes:\n", frequent_itemsets.head(60))


C:\Users\jmvdp\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Itemsets frequentes:
       support                       itemsets
0    0.357517                        (RMPDR)
300  0.260546                       (Outono)
294  0.252220                    (Primavera)
1    0.213843                      (Inverno)
28   0.196165                 (EOVRDJOPANVP)
167  0.094663                (Fim_de_Semana)
971  0.093418                (Outono, RMPDR)
962  0.085419             (Primavera, RMPDR)
301  0.079039               (Inverno, RMPDR)
437  0.053651      (Primavera, EOVRDJOPANVP)
438  0.050787         (EOVRDJOPANVP, Outono)
23   0.046473                        (RJPDR)
64   0.040009                           (CA)
14   0.039223                   (RNADRDDRSU)
145  0.036468                          (PED)
435  0.035453        (Inverno, EOVRDJOPANVP)
2    0.032819                          (DDP)
58   0.032662                        (ÁAORM)
90   0.031852                (LDVPDDPVELDVP)
11   0.031466                       (RSRPDP)
77   0.031043                    

In [8]:
# Generate association rules with lift metric and minimum threshold
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.01)
rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
print("Regras de associação:\n", rules.head(30))

Regras de associação:
                      antecedents                 consequents  \
669                (11063300701)                     (SOOLA)   
668                      (SOOLA)               (11063300701)   
1740         (RSRPDP, Primavera)               (11066601201)   
1741               (11066601201)         (RSRPDP, Primavera)   
1739    (11066601201, Primavera)                    (RSRPDP)   
1742                    (RSRPDP)    (11066601201, Primavera)   
246                (11066601201)                    (RSRPDP)   
247                     (RSRPDP)               (11066601201)   
923                (11066601004)                    (RSRPDP)   
922                     (RSRPDP)               (11066601004)   
835                (11066601208)                    (RSRPDP)   
834                     (RSRPDP)               (11066601208)   
1552               (11063301505)             (LDVPDDPVELDVP)   
1553             (LDVPDDPVELDVP)               (11063301505)   
1504          (Pr

In [16]:
#df = pd.read_csv("Merge_Gopi_With_Ipma_Precacumulada_Estacao.csv")
df.head(2)
chuva_frequency = df['Chuva'].max()

# Print the frequency
print(chuva_frequency)

True


In [9]:
# Filter rules involving 'Chuva'
rules_pressão_acumulada = rules[(rules['antecedents'].apply(lambda x: 'Chuva' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Chuva' in str(x)))]

print("rules_pressão_acumulada 'pressão_acumulada':\n", rules_pressão_acumulada.head(20))



rules_pressão_acumulada 'pressão_acumulada':
 Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []


In [8]:


# Filter rules involving 'Feriado'
rules_holiday = rules[(rules['antecedents'].apply(lambda x: 'Feriado' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Feriado' in str(x)))]

print("Regras que envolvem 'Feriado':\n", rules_holiday)

Regras que envolvem 'Feriado':
                      antecedents                 consequents  \
1393                   (Feriado)  (Primavera, Fim_de_Semana)   
1388  (Primavera, Fim_de_Semana)                   (Feriado)   
1389        (Primavera, Feriado)             (Fim_de_Semana)   
1392             (Fim_de_Semana)        (Primavera, Feriado)   
441                    (Feriado)             (Fim_de_Semana)   
440              (Fim_de_Semana)                   (Feriado)   
1390    (Fim_de_Semana, Feriado)                 (Primavera)   
1391                 (Primavera)    (Fim_de_Semana, Feriado)   
389                    (Feriado)                 (Primavera)   
388                  (Primavera)                   (Feriado)   
745                    (Feriado)                   (Inverno)   
744                    (Inverno)                   (Feriado)   
900               (EOVRDJOPANVP)                   (Feriado)   
901                    (Feriado)              (EOVRDJOPANVP)   
764     

In [16]:
# Filter rules involving 'Fim_de_Semana'
rules_pressão_acumulada = rules[(rules['antecedents'].apply(lambda x: 'Fim_de_Semana' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Fim_de_Semana' in str(x)))]

print("Fim_de_Semana':\n", rules_pressão_acumulada)


Fim_de_Semana':
                     antecedents                 consequents  \
557                   (Feriado)  (Primavera, Fim_de_Semana)   
552  (Primavera, Fim_de_Semana)                   (Feriado)   
554        (Primavera, Feriado)             (Fim_de_Semana)   
555             (Fim_de_Semana)        (Primavera, Feriado)   
295                   (Feriado)             (Fim_de_Semana)   
..                          ...                         ...   
246             (Fim_de_Semana)             (Outono, RMPDR)   
280  (Primavera, Fim_de_Semana)                     (RMPDR)   
285                     (RMPDR)  (Primavera, Fim_de_Semana)   
282          (Primavera, RMPDR)             (Fim_de_Semana)   
283             (Fim_de_Semana)          (Primavera, RMPDR)   

     antecedent support  consequent support   support  confidence      lift  \
557            0.008483            0.022488  0.001148    0.135328  6.017899   
552            0.022488            0.008483  0.001148    0.051048  6

In [11]:
subseccoes = ['11066601201', '11060200605', '11063301908', '11063301802', '11066600102',
    '11063301002', '11060200705', '11063302305', '11063300905', '11063302506',
    '11063300603', '11066601103', '11066601109', '11066601207', '11060201504',
    '11066600907', '11066601001', '11063300601', '11060200401', '11063302510',
    '11066601210', '11060200607', '11066600203', '11060200404', '11063302606',
    '11066600402', '11066600802', '11060200801', '11066600302', '11066600803',
    '11060200701', '11066601004', '11066600204', '11066600201', '11063302512',
    '11060201204', '11066601206', '11066600906', '11066601211', '11063301003',
    '11063300504', '11060201304', '11066601410', '11063302207', '11066600502',
    '11063302210', '11066601202', '11060200102', '11063301610', '11063300303',
    '11063301502', '11060201101', '11066600503', '11060200609', '11066601505',
    '11060200608', '11060201408', '11063302004', '11066601102', '11063301903',
    '11066601501', '11066600307', '11060201502', '11063302002', '11066601204',
    '11066601306', '11063301804', '11063302003', '11063301907', '11066600908',
    '11066601409', '11066600401', '11063301901', '11066600701', '11063301608',
    '11066601304', '11060200303', '11066600301', '11063302605', '11060200405',
    '11063300311', '11063300401', '11066601407', '11063300801', '11063302509',
    '11060200502', '11066601403', '11063300502', '11063301801', '11060201409',
    '11066601305', '11063301805', '11063301707', '11063301702', '11063300501',
    '11060201305', '11066601101', '11060201501', '11066600205', '11063302402',
    '11063300202', '11063302505', '11063302203', '11060201411', '11066600103',
    '11066600702', '11063301620', '11060200305', '11066600306', '11060200603',
    '11063301806', '11063301504', '11063302106', '11060200901', '11066600902',
    '11063301704', '11060200301', '11060200302', '11063301909', '11063301101',
    '11063301803', '11066601002', '11066600903', '11063300703', '11063300503',
    '11063301613', '11063300103', '11063301501', '11060201403', '11066600110',
    '11066600801', '11066601405', '11066600901', '11063302202', '11060201102',
    '11066600304', '11063301705', '11060200902', '11063302602', '11063301102',
    '11063300101', '11063301612', '11063300204', '11060201104', '11066601401',
    '11066600108', '11066600105', '11063300305', '11066600501', '11066600602',
    '11063301904', '11066600206', '11060200402', '11060201412', '11060201301',
    '11066601208', '11063302406', '11063302601', '11060200503', '11063301402',
    '11066601005', '11063302303', '11063301505', '11066600704', '11066601107',
    '11060201002', '11063302306', '11063300102', '11060201402', '11066601007',
    '11063301906', '11063302507', '11060200101', '11066600101', '11060200201',
    '11060200202', '11066601108', '11063301407', '11063301607', '11060200702',
    '11063300203', '11063301706', '11066601003', '11060200504', '11060201103',
    '11066601503', '11063302502', '11060201201', '11060200406', '11063300307',
    '11063300802', '11066601502', '11066601104', '11066600107', '11063300201',
    '11063300308', '11060200802', '11063302201', '11063302302', '11066600202',
    '11060200601', '11063300402', '11066600305', '11060201401', '11060201003',
    '11063300301', '11063301201', '11060200604', '11063301611', '11063301604',
    '11060201503', '11066601209', '11060201309', '11063302304', '11063301619',
    '11060201202', '11060201307', '11063301703', '11063300902', '11063300704',
    '11063301616', '11063300304', '11063302101', '11066600603', '11063300312',
    '11063302107', '11063300702', '11063300903', '11066601106', '11066600904',
    '11063302001', '11063301307', '11063302503', '11066600109', '11063302511',
    '11063301617', '11063301403', '11066600104', '11060200501', '11063300602',
    '11066601203', '11066600804', '11063301905', '11063301701', '11063301303',
    '11063302504', '11066601302', '11063301621', '11060200704', '11060201308',
    '11063302501', '11066601301', '11066601404', '11066600601', '11063301404',
    '11060201302', '11066601008', '11066600303', '11066601402', '11063302604',
    '11063301601', '11063301202', '11063301406', '11060200306', '11063302508',
    '11066601105', '11060200204', '11063301001', '11060200610', '11060200611',
    '11066601504', '11063300403', '11063300904', '11063302206', '11066601303',
    '11063301302', '11063300309', '11063301305', '11063300302', '11066600605',
    '11060200403', '11066601006', '11063301807', '11063302204', '11063302404',
    '11066600604', '11063300701', '11060200606', '11060200203', '11063301602',
    '11063302104', '11060200304', '11063301304', '11063301004', '11060201306',
    '11063302105', '11063302603', '11063301605', '11066600703', '11063302607',
    '11066601212', '11063302301']
rules_subseccoes = rules[rules['antecedents'].apply(lambda x: any(subseccao in str(x) for subseccao in subseccoes)) |
                      rules['consequents'].apply(lambda x: any(subseccao in str(x) for subseccao in subseccoes))]

print("Regras que envolvem estações:\n",rules_subseccoes.head(30))

Regras que envolvem estações:
                    antecedents               consequents  antecedent support  \
669              (11063300701)                   (SOOLA)            0.003105   
668                    (SOOLA)             (11063300701)            0.031043   
1740       (RSRPDP, Primavera)             (11066601201)            0.008048   
1741             (11066601201)       (RSRPDP, Primavera)            0.008785   
1739  (11066601201, Primavera)                  (RSRPDP)            0.002489   
1742                  (RSRPDP)  (11066601201, Primavera)            0.031466   
246              (11066601201)                  (RSRPDP)            0.008785   
247                   (RSRPDP)             (11066601201)            0.031466   
923              (11066601004)                  (RSRPDP)            0.004229   
922                   (RSRPDP)             (11066601004)            0.031466   
835              (11066601208)                  (RSRPDP)            0.004870   
834      

In [15]:
# Este código serve apenas para apresentar os dados de formna mais reduzida no capítulo 7 da dissertação

# Selecionar colunas principais
rules_selected = rules_subseccoes[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


# Exibir a tabela formatada
print("Regras que envolvem Subsecções (Top 25):\n")
print(rules_selected.head(25))

Regras que envolvem Subsecções (Top 25):

                   antecedents               consequents   support  \
669              (11063300701)                   (SOOLA)  0.001788   
668                    (SOOLA)             (11063300701)  0.001788   
1740       (RSRPDP, Primavera)             (11066601201)  0.001003   
1741             (11066601201)       (RSRPDP, Primavera)  0.001003   
1739  (11066601201, Primavera)                  (RSRPDP)  0.001003   
1742                  (RSRPDP)  (11066601201, Primavera)  0.001003   
246              (11066601201)                  (RSRPDP)  0.003444   
247                   (RSRPDP)             (11066601201)  0.003444   
923              (11066601004)                  (RSRPDP)  0.001474   
922                   (RSRPDP)             (11066601004)  0.001474   
835              (11066601208)                  (RSRPDP)  0.001571   
834                   (RSRPDP)             (11066601208)  0.001571   
1552             (11063301505)           (LDVPDD